In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import matplotlib.pyplot as plt
from pylab import figure, axes, pie, title, savefig
import time
import csv
import sys
import plotly.express as px

In [2]:
df = pd.read_csv("/Users/hamin/bithumb_bot_data/include/2018-new/2018-03-btc-krw.csv")

In [3]:
df.dtypes

timestamp     object
quantity     float64
price        float64
fee          float64
amount         int64
side         float64
dtype: object

In [4]:
## converting timestamp column type 'object' to 'datetime'
df['timestamp'] = pd.to_datetime(df['timestamp'], format='%Y-%m-%d %H:%M:%S', errors='raise')
df.dtypes

timestamp    datetime64[ns]
quantity            float64
price               float64
fee                 float64
amount                int64
side                float64
dtype: object

In [5]:
## converting timestamp to detailed date column
df['timestamp_year_month_day']       = df['timestamp'].dt.date         # YYYY-MM-DD(문자)
df['timestamp_year']       = df['timestamp'].dt.year         # 연(4자리숫자)
df['timestamp_month']      = df['timestamp'].dt.month        # 월(숫자)
df['timestamp_month_name'] = df['timestamp'].dt.month_name() # 월(문자)

df['timestamp_day']        = df['timestamp'].dt.day          # 일(숫자)
df['timestamp_time']       = df['timestamp'].dt.time         # HH:MM:SS(문자)
df['timestamp_hour']       = df['timestamp'].dt.hour         # 시(숫자)
df['timestamp_minute']     = df['timestamp'].dt.minute       # 분(숫자)
df['timestamp_second']     = df['timestamp'].dt.second       # 초(숫자)

In [6]:
df

,timestamp,quantity,price,fee,amount,side,timestamp_year_month_day,timestamp_year,timestamp_month,timestamp_month_name,timestamp_day,timestamp_time,timestamp_hour,timestamp_minute,timestamp_second
0,2018-03-01 00:11:00,0.9563,11929000.0,0.0,-11407703,0.0,2018-03-01,2018.0,3.0,March,1.0,00:11:00,0.0,11.0,0.0
1,2018-03-01 00:12:00,0.1422,11936000.0,0.0,-1697299,0.0,2018-03-01,2018.0,3.0,March,1.0,00:12:00,0.0,12.0,0.0
2,2018-03-01 00:12:00,0.0100,11941000.0,0.0,119410,1.0,2018-03-01,2018.0,3.0,March,1.0,00:12:00,0.0,12.0,0.0
3,2018-03-01 00:12:00,0.0032,11941000.0,0.0,38211,1.0,2018-03-01,2018.0,3.0,March,1.0,00:12:00,0.0,12.0,0.0
4,2018-03-01 00:12:00,0.1328,11940000.0,0.0,1585632,1.0,2018-03-01,2018.0,3.0,March,1.0,00:12:00,0.0,12.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33145,2018-03-31 23:59:00,0.1000,7796000.0,0.0,779600,1.0,2018-03-31,2018.0,3.0,March,31.0,23:59:00,23.0,59.0,0.0
33146,2018-03-31 23:59:00,0.0416,7797000.0,0.0,324355,1.0,2018-03-31,2018.0,3.0,March,31.0,23:59:00,23.0,59.0,0.0
33147,2018-03-31 23:59:00,0.0009,7794000.0,0.0,7015,1.0,2018-03-31,2018.0,3.0,March,31.0,23:59:00,23.0,59.0,0.0
33148,2018-03-31 23:59:00,0.3152,7794000.0,0.0,2456669,1.0,2018-03-31,2018.0,3.0,March,31.0,23:59:00,23.0,59.0,0.0


In [7]:
## creating 'day - amount' connection
group_day_amount = df['amount'].groupby(df['timestamp_year_month_day'])

In [8]:
group_day_amount.size()

timestamp_year_month_day
2018-03-01    1042
2018-03-02    1224
2018-03-03    1406
2018-03-04    1278
2018-03-05     613
2018-03-08    1049
2018-03-09    1028
2018-03-10    1119
2018-03-11     928
2018-03-12    1144
2018-03-13    1216
2018-03-14     731
2018-03-15    1081
2018-03-16     772
2018-03-17     666
2018-03-18    1046
2018-03-19    1346
2018-03-20    1744
2018-03-21    1706
2018-03-22    1782
2018-03-23    1178
2018-03-24    1389
2018-03-25    1176
2018-03-26    1149
2018-03-27    1341
2018-03-28    1240
2018-03-29     765
2018-03-30    1027
2018-03-31     963
Name: amount, dtype: int64

In [9]:
group_day_amount

In [10]:
## 일별 총 손익량
group_day_amount.sum()

timestamp_year_month_day
2018-03-01      842310
2018-03-02    -9850277
2018-03-03    12052822
2018-03-04   -17687024
2018-03-05    -2474659
2018-03-08   -11229029
2018-03-09    14113709
2018-03-10     1472564
2018-03-11     1168065
2018-03-12     1339762
2018-03-13      534832
2018-03-14      579270
2018-03-15     1668246
2018-03-16     -492516
2018-03-17   -19028555
2018-03-18    19884617
2018-03-19     7314411
2018-03-20     2737783
2018-03-21    -2746455
2018-03-22     5255741
2018-03-23      857685
2018-03-24   -13545062
2018-03-25    15396091
2018-03-26     -230397
2018-03-27   -15649586
2018-03-28    17142294
2018-03-29      381138
2018-03-30   -16682822
2018-03-31    19011544
Name: amount, dtype: int64

In [11]:
## creating 'day - earning amount' connection
df2 = df.copy()
df2 =df2[df2['amount'] > 0]
group_day_amount_positive = df2['amount'].groupby(df2['timestamp_year_month_day'])


In [12]:
# 일별 수익 transaction 수
group_day_amount_positive.size()

timestamp_year_month_day
2018-03-01     629
2018-03-02     706
2018-03-03     797
2018-03-04     708
2018-03-05     350
2018-03-08     597
2018-03-09     630
2018-03-10     670
2018-03-11     566
2018-03-12     666
2018-03-13     791
2018-03-14     449
2018-03-15     655
2018-03-16     511
2018-03-17     422
2018-03-18     661
2018-03-19     769
2018-03-20    1122
2018-03-21    1070
2018-03-22    1179
2018-03-23     774
2018-03-24     851
2018-03-25     689
2018-03-26     760
2018-03-27     834
2018-03-28     837
2018-03-29     512
2018-03-30     645
2018-03-31     603
Name: amount, dtype: int64

In [13]:
# 일별 수익량
group_day_amount_positive.sum()

timestamp_year_month_day
2018-03-01    1729078736
2018-03-02    2567799686
2018-03-03    2843235983
2018-03-04    2635126328
2018-03-05    1151553242
2018-03-08    2057855852
2018-03-09    2029628337
2018-03-10    2092984542
2018-03-11    1647154408
2018-03-12    2172887664
2018-03-13    2634083924
2018-03-14    1487680380
2018-03-15    2172246406
2018-03-16    1208244204
2018-03-17    1142256616
2018-03-18    1655232203
2018-03-19    2649213976
2018-03-20    3272418009
2018-03-21    3677377266
2018-03-22    3543600809
2018-03-23    2091273273
2018-03-24    2457649721
2018-03-25    2062421043
2018-03-26    1681001400
2018-03-27    1998762896
2018-03-28    2004241259
2018-03-29     979289919
2018-03-30    1667576198
2018-03-31    1357841319
Name: amount, dtype: int64

In [14]:
## creating 'day - loss amount' connection
df3 = df.copy()
df3 = df3[df3['amount'] < 0]
group_day_amount_negative = df3['amount'].groupby(df3['timestamp_year_month_day'])

In [15]:
# 일별 손해 transaction 수
group_day_amount_negative.size()

timestamp_year_month_day
2018-03-01    413
2018-03-02    518
2018-03-03    609
2018-03-04    570
2018-03-05    263
2018-03-08    452
2018-03-09    398
2018-03-10    449
2018-03-11    362
2018-03-12    478
2018-03-13    425
2018-03-14    282
2018-03-15    426
2018-03-16    261
2018-03-17    244
2018-03-18    385
2018-03-19    577
2018-03-20    622
2018-03-21    636
2018-03-22    603
2018-03-23    404
2018-03-24    538
2018-03-25    487
2018-03-26    389
2018-03-27    507
2018-03-28    403
2018-03-29    253
2018-03-30    382
2018-03-31    360
Name: amount, dtype: int64

In [16]:
# 일별 손해량
group_day_amount_negative.sum()

timestamp_year_month_day
2018-03-01   -1728236426
2018-03-02   -2577649963
2018-03-03   -2831183161
2018-03-04   -2652813352
2018-03-05   -1154027901
2018-03-08   -2069084881
2018-03-09   -2015514628
2018-03-10   -2091511978
2018-03-11   -1645986343
2018-03-12   -2171547902
2018-03-13   -2633549092
2018-03-14   -1487101110
2018-03-15   -2170578160
2018-03-16   -1208736720
2018-03-17   -1161285171
2018-03-18   -1635347586
2018-03-19   -2641899565
2018-03-20   -3269680226
2018-03-21   -3680123721
2018-03-22   -3538345068
2018-03-23   -2090415588
2018-03-24   -2471194783
2018-03-25   -2047024952
2018-03-26   -1681231797
2018-03-27   -2014412482
2018-03-28   -1987098965
2018-03-29    -978908781
2018-03-30   -1684259020
2018-03-31   -1338829775
Name: amount, dtype: int64

In [17]:
# 일별 절대값 총합
absolute_negative_sum=group_day_amount_negative.sum()
absolute_negative_sum = absolute_negative_sum * (-1)
absolute_total_amout=absolute_negative_sum+group_day_amount_positive.sum()
absolute_total_amout

timestamp_year_month_day
2018-03-01    3457315162
2018-03-02    5145449649
2018-03-03    5674419144
2018-03-04    5287939680
2018-03-05    2305581143
2018-03-08    4126940733
2018-03-09    4045142965
2018-03-10    4184496520
2018-03-11    3293140751
2018-03-12    4344435566
2018-03-13    5267633016
2018-03-14    2974781490
2018-03-15    4342824566
2018-03-16    2416980924
2018-03-17    2303541787
2018-03-18    3290579789
2018-03-19    5291113541
2018-03-20    6542098235
2018-03-21    7357500987
2018-03-22    7081945877
2018-03-23    4181688861
2018-03-24    4928844504
2018-03-25    4109445995
2018-03-26    3362233197
2018-03-27    4013175378
2018-03-28    3991340224
2018-03-29    1958198700
2018-03-30    3351835218
2018-03-31    2696671094
Name: amount, dtype: int64

In [18]:
## 일별 계산 table 생성
dfs = [group_day_amount.size(), group_day_amount.sum(), group_day_amount_positive.size(), group_day_amount_positive.sum(), group_day_amount_negative.size(), group_day_amount_negative.sum(), absolute_total_amout, absolute_negative_sum]
dfs

[timestamp_year_month_day
 2018-03-01    1042
 2018-03-02    1224
 2018-03-03    1406
 2018-03-04    1278
 2018-03-05     613
 2018-03-08    1049
 2018-03-09    1028
 2018-03-10    1119
 2018-03-11     928
 2018-03-12    1144
 2018-03-13    1216
 2018-03-14     731
 2018-03-15    1081
 2018-03-16     772
 2018-03-17     666
 2018-03-18    1046
 2018-03-19    1346
 2018-03-20    1744
 2018-03-21    1706
 2018-03-22    1782
 2018-03-23    1178
 2018-03-24    1389
 2018-03-25    1176
 2018-03-26    1149
 2018-03-27    1341
 2018-03-28    1240
 2018-03-29     765
 2018-03-30    1027
 2018-03-31     963
 Name: amount, dtype: int64,
 timestamp_year_month_day
 2018-03-01      842310
 2018-03-02    -9850277
 2018-03-03    12052822
 2018-03-04   -17687024
 2018-03-05    -2474659
 2018-03-08   -11229029
 2018-03-09    14113709
 2018-03-10     1472564
 2018-03-11     1168065
 2018-03-12     1339762
 2018-03-13      534832
 2018-03-14      579270
 2018-03-15     1668246
 2018-03-16     -492516
 20

In [19]:
day_df_final = reduce(lambda left,right: pd.merge(left,right,on='timestamp_year_month_day'), dfs)
day_df_final.columns = [ "daily transaction", "daily income", "daily Yield transaction", "daily Yield", "daily loss transaction", "daily loss", "daily absolute amount", "daily absolute negative amount"]
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-03-01,1042,842310,629,1729078736,413,-1728236426,3457315162,1728236426
2018-03-02,1224,-9850277,706,2567799686,518,-2577649963,5145449649,2577649963
2018-03-03,1406,12052822,797,2843235983,609,-2831183161,5674419144,2831183161
2018-03-04,1278,-17687024,708,2635126328,570,-2652813352,5287939680,2652813352
2018-03-05,613,-2474659,350,1151553242,263,-1154027901,2305581143,1154027901
2018-03-08,1049,-11229029,597,2057855852,452,-2069084881,4126940733,2069084881
2018-03-09,1028,14113709,630,2029628337,398,-2015514628,4045142965,2015514628
2018-03-10,1119,1472564,670,2092984542,449,-2091511978,4184496520,2091511978
2018-03-11,928,1168065,566,1647154408,362,-1645986343,3293140751,1645986343


In [20]:
print(day_df_final.asfreq('D'))

                          daily transaction  daily income  \
timestamp_year_month_day                                    
2018-03-01                           1042.0      842310.0   
2018-03-02                           1224.0    -9850277.0   
2018-03-03                           1406.0    12052822.0   
2018-03-04                           1278.0   -17687024.0   
2018-03-05                            613.0    -2474659.0   
2018-03-06                              NaN           NaN   
2018-03-07                              NaN           NaN   
2018-03-08                           1049.0   -11229029.0   
2018-03-09                           1028.0    14113709.0   
2018-03-10                           1119.0     1472564.0   
2018-03-11                            928.0     1168065.0   
2018-03-12                           1144.0     1339762.0   
2018-03-13                           1216.0      534832.0   
2018-03-14                            731.0      579270.0   
2018-03-15              

In [21]:
day_df_final.to_csv("/Users/hamin/bithumb_bot_data/2018-03-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')


In [22]:
# 뒤집기.
day_df_final[::-1].to_csv("/Users/hamin/bithumb_bot_data/2018-03-daily_result_table.csv", header=True, index=True, encoding = 'utf-8')

In [23]:
day_df_final

,daily transaction,daily income,daily Yield transaction,daily Yield,daily loss transaction,daily loss,daily absolute amount,daily absolute negative amount
timestamp_year_month_day,,,,,,,,
2018-03-01,1042,842310,629,1729078736,413,-1728236426,3457315162,1728236426
2018-03-02,1224,-9850277,706,2567799686,518,-2577649963,5145449649,2577649963
2018-03-03,1406,12052822,797,2843235983,609,-2831183161,5674419144,2831183161
2018-03-04,1278,-17687024,708,2635126328,570,-2652813352,5287939680,2652813352
2018-03-05,613,-2474659,350,1151553242,263,-1154027901,2305581143,1154027901
2018-03-08,1049,-11229029,597,2057855852,452,-2069084881,4126940733,2069084881
2018-03-09,1028,14113709,630,2029628337,398,-2015514628,4045142965,2015514628
2018-03-10,1119,1472564,670,2092984542,449,-2091511978,4184496520,2091511978
2018-03-11,928,1168065,566,1647154408,362,-1645986343,3293140751,1645986343
